# Exploratory: Check column null percentage

Summary: <br>
This notebook is used to explore the SDG&E weather station data that was extracted via Synopic API.

Open Questions: <br>

In [1]:
import pandas as pd

### Read in data

In [2]:
%%time
weather_df = pd.read_csv("../../data/raw/weather_sdge.csv", header=[0, 1])

CPU times: user 55.1 s, sys: 11.8 s, total: 1min 6s
Wall time: 1min 6s


### Calculate attribute null percentage across all stations

In [3]:
rows = weather_df.shape[0]
weather_df.shape

(45797111, 23)

In [4]:
%%time
pd.DataFrame(weather_df.isna().sum() / rows * 100, columns=["null_percentage"])

CPU times: user 14 s, sys: 276 ms, total: 14.3 s
Wall time: 14.3 s


,,null_percentage
Station_ID,Unnamed: 0_level_1,0.000000
Date_Time,Unnamed: 1_level_1,0.000000
air_temp_set_1,Celsius,0.318103
relative_humidity_set_1,%,0.280882
wind_speed_set_1,m/s,0.029757
wind_direction_set_1,Degrees,4.179626
wind_gust_set_1,m/s,0.024185
volt_set_1,volts,63.005819
solar_radiation_set_1,W/m**2,98.297144
dew_point_temperature_set_1d,Celsius,0.322815


## Calculate null percentage per station

### Generate null counts for each column for each station

In [5]:
tmp = weather_df.set_index("Station_ID")

In [6]:
station_df = tmp.isna().groupby("Station_ID").sum()

# update index
# just take the station value in each list
orig_index = station_df.index.to_list()
new_index = [item[0] for item in orig_index]
station_df.index = pd.Index(new_index, name="Station_ID")

station_df

Date_Time air_temp_set_1 relative_humidity_set_1  \
           Unnamed: 1_level_1        Celsius                       %   
Station_ID                                                             
AMOSD                       0            684                     595   
ANESD                       0              0                       0   
ARHSD                       0              0                       3   
AVOSD                       0              0                       1   
AVYSD                       0           3891                       0   
...                       ...            ...                     ...   
WSTSD                       0              0                       0   
WSYSD                       0              0                       1   
WWYSD                       0              0                       1   
WYNSD                       0              0                       0   
YSASD                       0              0                       0   

           wind_speed_set_1 wind_direction_set_1 wind_gust_set_1 volt_set_1  \
                        m/s              Degrees             m/s      volts   
Station_ID                                                                    
AMOSD                    70                10574               0     167787   
ANESD                    52                57261               0     183399   
ARHSD                    74                22629               0     168422   
AVOSD                    58                22961               0     175839   
AVYSD                    83                 2957               0     176702   
...                     ...                  ...             ...        ...   
WSTSD                    37                   68               0       1282   
WSYSD                    36                 5271               0     204646   
WWYSD                    66                 6352               0     148045   
WYNSD                    72                 3959               0     175698   
YSASD                    46                33379               0     205656   

           solar_radiation_set_1 dew_point_temperature_set_1d  \
                          W/m**2                      Celsius   
Station_ID                                                      
AMOSD                     261384                         1279   
ANESD                     255017                            0   
ARHSD                     264358                            3   
AVOSD                     267678                            1   
AVYSD                     267714                         3891   
...                          ...                          ...   
WSTSD                      96405                            0   
WSYSD                     266658                            1   
WWYSD                     238705                            1   
WYNSD                     267613                            0   
YSASD                     267639                            0   

           wind_chill_set_1d  ... altimeter_set_1 pressure_set_1  \
                     Celsius  ...         Pascals        Pascals   
Station_ID                    ...                                  
AMOSD                 261266  ...          261384         261384   
ANESD                 267099  ...          267102         267102   
ARHSD                 262961  ...          264358         264358   
AVOSD                 267631  ...          267678         267678   
AVYSD                 261746  ...          267714         267714   
...                      ...  ...             ...            ...   
WSTSD                  87232  ...           96405          96405   
WSYSD                 259500  ...          266658         266658   
WWYSD                 236712  ...          238705         238705   
WYNSD                 259486  ...          267613         267613   
YSASD                 267052  ...          267639         267639   

           pressure_set_1d sea_level_pre

### Generate total row counts for each station

In [7]:
counts_df = (
    tmp["Date_Time"]
    .groupby("Station_ID")
    .count()
    .rename(columns={"Unnamed: 1_level_1": "row_count"})
)

# update index
# just take the station value in each list
orig_index = counts_df.index.to_list()
new_index = [item[0] for item in orig_index]
counts_df.index = pd.Index(new_index, name="Station_ID")

counts_df

,row_count
Station_ID,
AMOSD,261384
ANESD,267102
ARHSD,264358
AVOSD,267678
AVYSD,267714
...,...
WSTSD,96405
WSYSD,266658
WWYSD,238705


### Calculate null percentage for each attribute of each station

In [8]:
station_null_per_df = station_df.copy()

In [9]:
# station_null_per_df.loc["AMOSD"]

In [10]:
for station in counts_df.index:
    # print(station)
    # print(station_null_per_df.loc[station] / counts_df.loc[station]["row_count"])
    # print(counts_df.loc[station]["row_count"])
    station_null_per_df.loc[station] = (
        station_null_per_df.loc[station] / counts_df.loc[station]["row_count"] * 100
    )

In [11]:
station_null_per_df
# station_null_per_df.to_csv("station_null_per.csv")

Date_Time air_temp_set_1 relative_humidity_set_1  \
           Unnamed: 1_level_1        Celsius                       %   
Station_ID                                                             
AMOSD                       0       0.261684                0.227634   
ANESD                       0       0.000000                0.000000   
ARHSD                       0       0.000000                0.001135   
AVOSD                       0       0.000000                0.000374   
AVYSD                       0       1.453417                0.000000   
...                       ...            ...                     ...   
WSTSD                       0       0.000000                0.000000   
WSYSD                       0       0.000000                0.000375   
WWYSD                       0       0.000000                0.000419   
WYNSD                       0       0.000000                0.000000   
YSASD                       0       0.000000                0.000000   

           wind_speed_set_1 wind_direction_set_1 wind_gust_set_1 volt_set_1  \
                        m/s              Degrees             m/s      volts   
Station_ID                                                                    
AMOSD              0.026781             4.045389             0.0  64.191764   
ANESD              0.019468            21.437878             0.0  68.662533   
ARHSD              0.027992             8.559983             0.0  63.709818   
AVOSD              0.021668             8.577844             0.0  65.690494   
AVYSD              0.031003             1.104537             0.0  66.004019   
...                     ...                  ...             ...        ...   
WSTSD              0.038380             0.070536             0.0   1.329807   
WSYSD              0.013500             1.976689             0.0  76.744744   
WWYSD              0.027649             2.661025             0.0  62.020067   
WYNSD              0.026905             1.479375             0.0  65.653761   
YSASD              0.017187            12.471650             0.0  76.840819   

           solar_radiation_set_1 dew_point_temperature_set_1d  \
                          W/m**2                      Celsius   
Station_ID                                                      
AMOSD                 100.000000                     0.489318   
ANESD                  95.475511                     0.000000   
ARHSD                 100.000000                     0.001135   
AVOSD                 100.000000                     0.000374   
AVYSD                 100.000000                     1.453417   
...                          ...                          ...   
WSTSD                 100.000000                     0.000000   
WSYSD                 100.000000                     0.000375   
WWYSD                 100.000000                     0.000419   
WYNSD                 100.000000                     0.000000   
YSASD                 100.000000                     0.000000   

           wind_chill_set_1d  ... altimeter_set_1 pressure_set_1  \
                     Celsius  ...         Pascals        Pascals   
Station_ID                    ...                                  
AMOSD              99.954856  ...             100            100   
ANESD              99.998877  ...             100            100   
ARHSD              99.471550  ...             100            100   
AVOSD              99.982442  ...             100            100   
AVYSD              97.770755  ...             100            100   
...                      ...  ...             ...            ...   
WSTSD              90.484933  ...             100            100   
WSYSD              97.315663  ...             100            100   
WWYSD              99.165078  ...             100            100   
WYNSD              96.963152  ...             100            100   
YSASD              99.780675  ...             100            100   

           pressure_set_1d sea_level_pre

## Check wind direction nulls

Is wind direction null only when wind speed is 0? -> Primarily yes. Wind speed=0 accounts for 99.99% of null wind directions.

In [12]:
weather_df[weather_df["wind_direction_set_1"]["Degrees"].isna()]

Station_ID             Date_Time air_temp_set_1  \
         Unnamed: 0_level_1    Unnamed: 1_level_1        Celsius   
17                    ANESD  2016-06-03T02:50:00Z         26.110   
18                    ANESD  2016-06-03T03:00:00Z         25.560   
19                    ANESD  2016-06-03T03:10:00Z         25.000   
20                    ANESD  2016-06-03T03:20:00Z         23.890   
21                    ANESD  2016-06-03T03:30:00Z         23.890   
...                     ...                   ...            ...   
45794384              BUBSD  2021-06-21T10:10:00Z         21.289   
45794510              BUBSD  2021-06-22T07:10:00Z         23.622   
45795079              BUBSD  2021-06-26T07:10:00Z         23.044   
45795931              BUBSD  2021-07-02T07:10:00Z         25.272   
45796185              BUBSD  2021-07-04T08:00:00Z         25.100   

         relative_humidity_set_1 wind_speed_set_1 wind_direction_set_1  \
                               %              m/s              Degrees   
17                         43.00              0.0                  NaN   
18                         45.00              0.0                  NaN   
19                         47.00              0.0                  NaN   
20                         49.00              0.0                  NaN   
21                         52.00              0.0                  NaN   
...                          ...              ...                  ...   
45794384                   22.58              0.0                  NaN   
45794510                   31.89              0.0                  NaN   
45795079                   38.86              0.0                  NaN   
45795931                   41.71              0.0                  NaN   
45796185                   20.18              0.0                  NaN   

         wind_gust_set_1 volt_set_1 solar_radiation_set_1  \
                     m/s      volts                W/m**2   
17                  0.45        NaN                   NaN   
18                  0.45        NaN                   NaN   
19                  0.45        NaN                   NaN   
20                  0.45        NaN                   NaN   
21                  0.45        NaN                   NaN   
...                  ...        ...                   ...   
45794384            0.00      12.74                   NaN   
45794510            0.00      12.81                   NaN   
45795079            0.00      12.81                   NaN   
45795931            0.00      12.81                   NaN   
45796185            0.00      12.79                   NaN   

         dew_point_temperature_set_1d  ... altimeter_set_1 pressure_set_1  \
                              Celsius  ...         Pascals        Pascals   
17                              12.54  ...             NaN            NaN   
18                              12.74  ...             NaN            NaN   
19                              12.89  ...             NaN            NaN   
20                              12.50  ...             NaN            NaN   
21                              13.41  ...             NaN            NaN   
...                               ...  ...             ...            ...   
45794384                        -0.97  ...             NaN            NaN   
45794510                         5.88  ...             NaN            NaN   
45795079                         8.25  ...             NaN            NaN   
45795931                        11.32  ...             NaN            NaN   
45796185                         0.70  ...             NaN            NaN   

         pressure_set_1d sea_level_pressure_set_1d            \
                 Pascals                   Pascals Pascals.1   
17                   NaN                       NaN       NaN   
18                   NaN                       NaN       NaN   
19                   NaN                       NaN       NaN   
20                   NaN                       NaN   

In [13]:
weather_df[weather_df["wind_direction_set_1"]["Degrees"].isna()]["wind_speed_set_1"][
    "m/s"
].value_counts(normalize=True) * 100

0.000    99.989238
0.468     0.000157
0.980     0.000104
0.226     0.000104
0.911     0.000104
           ...    
1.550     0.000052
2.710     0.000052
0.093     0.000052
1.173     0.000052
2.768     0.000052
Name: m/s, Length: 183, dtype: float64